In [6]:
!%pip install yfinance

zsh:fg:1: no job control in this shell.


# IMPORT LIBRARIES

In [8]:
import pandas as pd
import yfinance as yf
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

 Download 5 years historical daily price data for the Johannesburg Stock Exchange index, for four large companies of your choice on the JSE, as well as another financial variable of your choice. From the prices determine the daily returns.
Build a factor model for these returns. Give both the model with the factor loadings and the components (factors) that you include in the model.

In [9]:
stocks_list =['MTN.JO','ABSP.JO','NED.JO','VOD.JO',"ZAR=X"]
#Get Data from Yahoo Finance
data=yf.download(stocks_list,start='2018-2-01', end='2023-01-31')['Adj Close']
data['ZAR=X'] = data['ZAR=X'] *100 #convert exchange rate to cents
print(data.head(10))
data.shape

[*********************100%%**********************]  5 of 5 completed


Ticker           ABSP.JO        MTN.JO        NED.JO        VOD.JO   
Date                                                                 
2018-02-01  38752.707031  10544.688477  18686.710938  11809.777344  \
2018-02-02  38754.441406  10433.271484  18614.085938  11656.468750   
2018-02-05  38810.457031   9975.672852  18035.216797  11374.575195   
2018-02-06  38752.707031   9812.529297  18026.048828  11250.938477   
2018-02-07  38752.707031  10252.619141  18955.347656  11371.042969   
2018-02-08  38839.335938  10083.109375  18968.740234  11381.639648   
2018-02-09  38781.582031   9927.127930  19389.675781  11202.189453   
2018-02-12  38783.312500  10027.401367  19389.675781  11479.136719   
2018-02-13  38868.207031  10332.204102  19558.187500  11525.766602   
2018-02-14  38752.707031  10226.358398  19460.888672  11684.726562   

Ticker            ZAR=X  
Date                     
2018-02-01  1183.839989  
2018-02-02  1183.619976  
2018-02-05  1206.077003  
2018-02-06  1210.490036  
201

(1303, 5)

# ESTIMATE DAILY RETURNS

In [10]:
returns=np.log(data).diff() #daily returns
returns.head()

Ticker,ABSP.JO,MTN.JO,NED.JO,VOD.JO,ZAR=X
Date,,,,,
2018-02-01,NaN,NaN,NaN,NaN,NaN
2018-02-02,0.000045,-0.010622,-0.003894,-0.013066,-0.000186
2018-02-05,0.001444,-0.044850,-0.031592,-0.024481,0.018795
2018-02-06,-0.001489,-0.016489,-0.000508,-0.010929,0.003652
2018-02-07,0.000000,0.043873,0.050268,0.010618,-0.014655


# DATA CLEANING

In [11]:
#Drop rows with NaN values
returns_cleaned = returns.dropna()
returns_cleaned.shape
returns_cleaned

Ticker,ABSP.JO,MTN.JO,NED.JO,VOD.JO,ZAR=X
Date,,,,,
2018-02-02,0.000045,-0.010622,-0.003894,-0.013066,-0.000186
2018-02-05,0.001444,-0.044850,-0.031592,-0.024481,0.018795
2018-02-06,-0.001489,-0.016489,-0.000508,-0.010929,0.003652
2018-02-07,0.000000,0.043873,0.050268,0.010618,-0.014655
2018-02-08,0.002233,-0.016672,0.000706,0.000931,0.010953
...,...,...,...,...,...
2023-01-24,-0.009180,0.006329,0.000622,0.008964,0.003545
2023-01-25,0.000035,0.006506,-0.006959,-0.005858,-0.008356
2023-01-26,0.040116,0.026240,0.014973,-0.008605,0.006173


# MEAN RETURNS 

In [12]:
#Calculate the mean returns for each stock
# mean_returns = returns_cleaned.mean()
mean_returns=np.mean(returns_cleaned, axis=0)
mean_returns

Ticker
ABSP.JO    0.000714
MTN.JO     0.000199
NED.JO     0.000090
VOD.JO     0.000092
ZAR=X      0.000193
dtype: float64

# CENTER DATA

In [13]:
#Center the data by subtracting the mean returns
centered_data = returns_cleaned - mean_returns
centered_data

Ticker,ABSP.JO,MTN.JO,NED.JO,VOD.JO,ZAR=X
Date,,,,,
2018-02-02,-0.000670,-0.010822,-0.003984,-0.013158,-0.000379
2018-02-05,0.000730,-0.045050,-0.031682,-0.024572,0.018602
2018-02-06,-0.002204,-0.016689,-0.000599,-0.011021,0.003459
2018-02-07,-0.000714,0.043674,0.050178,0.010527,-0.014848
2018-02-08,0.001519,-0.016871,0.000616,0.000840,0.010760
...,...,...,...,...,...
2023-01-24,-0.009894,0.006130,0.000532,0.008872,0.003352
2023-01-25,-0.000679,0.006306,-0.007049,-0.005949,-0.008550
2023-01-26,0.039402,0.026040,0.014883,-0.008696,0.005980


# COVARIANCE MATRIX

In [14]:
#Compute the covariance matrix
n=returns_cleaned.shape[0]
cov_matrix = np.dot(centered_data.T, centered_data) / (n - 1)
cov_matrix

array([[ 1.77214675e-04,  6.66552619e-05,  5.38091073e-05,
        -2.62654120e-07, -4.44351645e-06],
       [ 6.66552619e-05,  8.71477316e-04,  3.25775513e-04,
         3.85086584e-04, -1.91352754e-05],
       [ 5.38091073e-05,  3.25775513e-04,  6.32453035e-04,
         2.06665865e-04, -3.31482402e-05],
       [-2.62654120e-07,  3.85086584e-04,  2.06665865e-04,
         3.57318576e-02, -9.13189002e-05],
       [-4.44351645e-06, -1.91352754e-05, -3.31482402e-05,
        -9.13189002e-05,  9.55779941e-05]])

In [15]:
#Check Covariance with Blackbox
cov_matrix=returns_cleaned.cov()
cov_matrix

Ticker,ABSP.JO,MTN.JO,NED.JO,VOD.JO,ZAR=X
Ticker,,,,,
ABSP.JO,1.772147e-04,0.000067,0.000054,-2.626541e-07,-0.000004
MTN.JO,6.665526e-05,0.000871,0.000326,3.850866e-04,-0.000019
NED.JO,5.380911e-05,0.000326,0.000632,2.066659e-04,-0.000033
VOD.JO,-2.626541e-07,0.000385,0.000207,3.573186e-02,-0.000091
ZAR=X,-4.443516e-06,-0.000019,-0.000033,-9.131890e-05,0.000096


# Eigenvalues and eigenvectors

In [16]:
#Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eigenvalues

array([3.57376064e-02, 1.10257042e-03, 4.05883895e-04, 1.69178684e-04,
       9.33412584e-05])

In [17]:
eigenvectors

array([[ 2.28115043e-05, -9.20788898e-02,  2.45764299e-02,
        -9.95310741e-01,  1.65531157e-02],
       [ 1.11012465e-02, -8.14443718e-01, -5.76905178e-01,
         6.10431472e-02, -3.51334702e-03],
       [ 5.99204852e-03, -5.71767953e-01,  8.14768281e-01,
         7.40284539e-02,  6.09749062e-02],
       [ 9.99917114e-01,  1.25569216e-02,  1.38739987e-03,
        -1.06721749e-03,  2.24171824e-03],
       [-2.57343613e-03,  3.35654722e-02, -5.22220066e-02,
         1.22029613e-02,  9.97993330e-01]])

# SORT EIGENVECTORS AND EIGENVALUES 

In [18]:
# Sort eigenvalues and eigenvectors in descending order
sorted_variables = np.argsort(eigenvalues)[::-1]
eigenvalues= eigenvalues[sorted_variables]
eigenvectors= eigenvectors[:, sorted_variables]

# PRINCIPAL COMPONENTS 

In [19]:
# Principal Components
PC1=eigenvectors[:, 0]
PC2=eigenvectors[:, 1]
PC3=eigenvectors[:, 2]
PC4=eigenvectors[:, 3]
PC5=eigenvectors[:, 4]
#Principal Values
PV1=eigenvalues[0]
PV2=eigenvalues[1]
PV3=eigenvalues[2]
PV4=eigenvalues[3]
PV5=eigenvalues[4]
#Print factors (principal components)
print("Factors","F1",PC1, "\n F2",PC2,"\n F3",PC3,"\n F4",PC4,"\n F5",PC5)

Factors F1 [ 2.28115043e-05  1.11012465e-02  5.99204852e-03  9.99917114e-01
 -2.57343613e-03] 
 F2 [-0.09207889 -0.81444372 -0.57176795  0.01255692  0.03356547] 
 F3 [ 0.02457643 -0.57690518  0.81476828  0.0013874  -0.05222201] 
 F4 [-0.99531074  0.06104315  0.07402845 -0.00106722  0.01220296] 
 F5 [ 0.01655312 -0.00351335  0.06097491  0.00224172  0.99799333]


# Financial assets that contribute most to the first principal component

Analysing the first principal component it can be observed that the component in the vector that has MTN and Vodacom has the highest values and therefore the assets that contribute most to the first principal component are Vodacom and MTN. 